# Investigating Borrower Reliability

Our client is the crediting department of a bank. They want to know if a bank client's family status and number of children affects whether they pay off their loan on time. We are given data on loan repayments from a number of the bank's clients.

The results of our study will be used when creating a **credit scoring** system: a system that the bank will use to estimate a potential borrower's ability to pay off their loan.

We need to use to data to answer three questions for the client:

1. Is there a connection between having children and paying off loans on time?
2. Is there a connection between marital status and paying off loans on time?
3. Is there a connection between income level and paying off loans on time?
4. Is there a connection between loan purpose and paying off loans on time?

## Data Overview

### Libraries

In [1]:
#pandas for dataframes
import pandas as pd

#Mystem for Russian lemmatization 
from pymystem3 import Mystem

#immediately generate a Mystem object
m = Mystem()

### Importing Data

Let's take a look at our data.

In [2]:
data = pd.read_csv('D://Documents/Courses/Yandex.Practicum - Data Science/Projects/2 - Scoring System/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Conclusions

The dataframe was imported successfully, but needs work. These are missing values, strange values in existing columns, and possible duplicates.

## Data Preprocessing

### Missing Values

Let's see how many missing entries we have.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

That's noticeable number of missing entries, a whole 10% of the table. They're all in the `days_employed` and `total_income` columns, too. Perhaps they're all unemployed? Let's take a closer look at `days_employed` first.

<div class="alert alert-success">
<b>✅ Комментарий ревьюера :</b> 

Так же старайся делать оценку доли пропусков - она более информативна, для этого можешь прописать `mean` вместо `sum`
</div>

In [7]:
data[data['days_employed'].isna()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


These are not all unemployed, so that hypothesis did not pan out.

We don't know why there are missing values in `days_employed`, since we can't ask our data source. With this absence of information, we're going to have to make a decision to fill in ouor missing values ourselves. To do this, we will replace them with the median value of their `income_type` group.

But first, let's take a look at `total_income`.

In [8]:
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [9]:
data[data['total_income'].isna() & data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


The missing entries for `days_employed` and `total_income` are all on the same lines, so we can fix them with one function.

For both columns, we will be replacing the missing values with the median value for their `income_type`.

In [10]:
#see what income_type categories exist
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [11]:
#make a list of the most common income_type categories
inc_types = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий']

In [12]:
#replace the missing values in total_income and days_employed with the median values of each category
for inc_type in inc_types:
    
    #total_income median
    med_total = data[data['income_type'] == inc_type]['total_income'].median()
    
    #days_employed median
    med_days = data[data['income_type'] == inc_type]['days_employed'].median()
    
    #rewrite the missing values
    data[data['income_type'] == inc_type] = data[data['income_type'] == inc_type].fillna(value={'total_income' : med_total})
    data[data['income_type'] == inc_type] = data[data['income_type'] == inc_type].fillna(value={'days_employed' : med_days})

Let's check the results.

In [13]:
data.isna().sum()

children            0
days_employed       1
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

That leaves us with a single missing value that we can just delete.

In [14]:
data = data.dropna()
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

While we're here, let's take a look at the `children` column, since it's important to our hypothesis testing.

In [15]:
data['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Whoa, negative children? And 20 children, when we don't have 19 or 21? We need to fix this.

The question is once again, where does this data come from. It's possible these were data entry errors, with the intent being 1 child (instead of -1) and 2 children (instead of 20). It could also be something else. In either case, these are 123 lines out of 20,000, so we can safely drop them.

In [16]:
#drop negative children
data = data[data['children'] >= 0]

#get rid of children=20
data = data[data['children'] != 20]

#check results
data['children'].value_counts()

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Excellent, the children are fixed.

#### Conclusion

Obvious missing values in `days_employed` and `total_income` were replaced with median values for the appropriate income types. Lines with an abnormal number of children (-1, 20) were dropped.

### Data Types

Let's take another look at the data types in our table.

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21401 non-null  int64  
 1   days_employed     21401 non-null  float64
 2   dob_years         21401 non-null  int64  
 3   education         21401 non-null  object 
 4   education_id      21401 non-null  int64  
 5   family_status     21401 non-null  object 
 6   family_status_id  21401 non-null  int64  
 7   gender            21401 non-null  object 
 8   income_type       21401 non-null  object 
 9   debt              21401 non-null  int64  
 10  total_income      21401 non-null  float64
 11  purpose           21401 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


These are not ideal. `days_employed` is listed as float, which is unnecessary. Let's rewrite that as int.

In [18]:
data['days_employed'] = data['days_employed'].astype('int64')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21401 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21401 non-null  int64  
 1   days_employed     21401 non-null  int64  
 2   dob_years         21401 non-null  int64  
 3   education         21401 non-null  object 
 4   education_id      21401 non-null  int64  
 5   family_status     21401 non-null  object 
 6   family_status_id  21401 non-null  int64  
 7   gender            21401 non-null  object 
 8   income_type       21401 non-null  object 
 9   debt              21401 non-null  int64  
 10  total_income      21401 non-null  float64
 11  purpose           21401 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


Good. Now let's see the result in the table itself.

In [20]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Whoa, why do we have negative `days_employed`? That doesn't work at all.

We once again don't know the exact reason behind this error, though it's likely either a data entry problem, or an unsigned integer error. Either way, let's rewrite `days_employed` as an absolute value.

In [21]:
data['days_employed'] = data['days_employed'].apply(abs)

In [22]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Conclusion

We changed the `days_employed` data type to int64 and used its absolute value.

### Duplicates

Let's take a look at blatant duplicates.

In [23]:
data.duplicated().sum()

54

Let's see what it defines as a duplicate.

In [24]:
data[data.duplicated(keep=False)].sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19321,0,1574,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594.396847,сделка с подержанным автомобилем
15892,0,1574,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594.396847,сделка с подержанным автомобилем
18328,0,1574,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594.396847,покупка жилой недвижимости
3452,0,1574,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594.396847,покупка жилой недвижимости
8629,1,1574,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,покупка коммерческой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
13639,0,365213,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,автомобиль
3609,0,365213,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,жилье
12389,0,365213,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,дополнительное образование
5865,0,365213,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514.486412,операции со своей недвижимостью


We've come to an interesting point. Yes, we can see the duplicates, but without something to uniquely identify specific clients like an ID or at least a name, we can't definitively say that these are duplicates. After all, it is possible that two 66-year old pensioners took out a credit for real estate purposes in the same year.

Also note that our duplicates have the median values for `days_employed` and `total_income`. We don't know how important this fact is at the moment.

Just in case, let's convert all information in `education`, `family_status`, `income_type`, and `purpose` to lowercase to ensure we got all the duplicates we can.

In [25]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

In [26]:
data.duplicated().sum()

71

At the end of the day, that's 71 duplicates out of 20,000 entries. We can safely delete them.

In [27]:
data = data.drop_duplicates().reset_index(drop=True)

Now let's take a look at `dob_years`.

In [28]:
data['dob_years'].value_counts()

35    613
40    601
41    601
34    595
38    594
42    591
33    577
39    571
31    555
36    552
29    542
44    541
48    534
30    533
37    530
43    509
50    508
49    505
32    505
28    501
45    493
27    490
52    483
56    478
47    477
54    473
46    466
53    457
57    453
58    453
51    444
55    441
59    440
26    406
60    373
25    356
61    352
62    347
63    268
24    263
64    258
23    250
65    193
22    183
66    182
67    167
21    110
0     100
68     99
69     83
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Whoa, I don't think the bank is supposed to give out loans to newborns. There aren't that many entries here (100 out of 20,000), so we will drop them as well. 

In [29]:
data = data[data['dob_years'] != 0]

#### Conclusion

71 duplicate lines were removed. 100 entries with `dob_years` of 0 were dropped.

### Lemmatization and Data Categorization

When looking at our dataframe, we can notice another possible source of duplicates. The `purpose` column can have several options that mean the same thing. Thus, we need to bring it to a more standard form.

First, let's take a look at the variants we currently have.

In [30]:
data['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                598
на покупку своего автомобиля              501
заняться высшим образованием      

We can break these up into four general groups:

1. marriage ("свадьба")
2. real estate ("недвижимость")
3. car ("автомобиль")
4. education ("образование")

Let's create two new columns: `purpose_categorized`, which will contain the purpose category; and `purpose_id`, which will have one of the following IDs:

0 - wedding

1 - real estate

2 - car

3 - education

99 - error

To do this, we will lemmatize the contents of `purpose`, search through the resulting lemmas for keywords, and assign the appropriate ID.

In [31]:
#create a category based on lemmas
def purpose_categorizer(purpose):
    
    #lemmatize the input
    lemmas = m.lemmatize(purpose)
    
    #look for keywords
    if 'свадьба' in lemmas:
        return 'wedding'
    
    elif ('недвижимость' in lemmas or 'жилье' in lemmas):
        return 'real estate'
    
    elif 'автомобиль' in lemmas:
        return 'car'
    
    elif 'образование' in lemmas:
        return 'education'
    
    #error code
    else:
        return 'error'

In [32]:
#create a purpose ID based on the category in purpose_categorized
def purpose_id_creator(p):
    if p == 'wedding':
        return 0
    elif p == 'real estate':
        return 1
    elif p == 'car':
        return 2
    elif p == 'education':
        return 3
    else:
        return 99

Let's see what our category breakdown is like.

In [33]:
%%time
data['purpose_categorized'] = data['purpose'].apply(purpose_categorizer)
data['purpose_categorized'].value_counts()

CPU times: total: 1min 21s
Wall time: 6h 2min 24s


real estate    10703
car             4258
education       3970
wedding         2299
Name: purpose_categorized, dtype: int64

This category split is fine for our needs.

While we're here, let's make a function that will create a `work_id` from `income_type`.

First, let's take a look at the income type distribution.

In [34]:
data['income_type'].value_counts()

сотрудник          10961
компаньон           5027
пенсионер           3792
госслужащий         1445
безработный            2
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

That's a fair amount of types. Let's now write our function.

In [35]:
def work_id_creator(inc):
    if inc == 'сотрудник':
        return 0
    elif inc == 'компаньон':
        return 1
    elif inc == 'пенсионер':
        return 2
    elif inc == 'госслужащий':
        return 3
    elif inc == 'предприниматель':
        return 4
    elif inc == 'безработный':
        return 5
    elif inc == 'в декрете':
        return 6
    else:
        return 99

And let's launch our functions to generate new columns.

In [36]:
#launch functions
data['purpose_id'] = data['purpose_categorized'].apply(purpose_id_creator)
data['work_id'] = data['income_type'].apply(work_id_creator)

#double-check results
display(data['purpose_categorized'].value_counts())
display(data['purpose_id'].value_counts())
data['work_id'].value_counts()

real estate    10703
car             4258
education       3970
wedding         2299
Name: purpose_categorized, dtype: int64

1    10703
2     4258
3     3970
0     2299
Name: purpose_id, dtype: int64

0     10961
1      5027
2      3792
3      1445
5         2
99        1
4         1
6         1
Name: work_id, dtype: int64

In [37]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categorized,purpose_id,work_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,real estate,1,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,car,2,0
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,real estate,1,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,education,3,0
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,wedding,0,2


Excellent! Now, let's start moving unnecessary columns out of our table. To start with, let's create the following dictionaries:

* `purpose` (to `purpose_id`),
* `income type` (to `work_id`),
* `education` (to `education_id`),
* `family_status` (to `family_id`).

In [38]:
#create the dictionary
purpose_dict = data[['purpose_categorized', 'purpose_id']]

#delete the duplicates and sort
purpose_dict = purpose_dict.drop_duplicates().sort_values(by='purpose_id').reset_index(drop=True)
purpose_dict

,purpose_categorized,purpose_id
0,wedding,0
1,real estate,1
2,car,2
3,education,3


In [39]:
#create the dictionary
education_dict = data[['education', 'education_id']]

#delete the duplicates and sort
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [40]:
#create the dictionary
family_dict = data[['family_status', 'family_status_id']]

#delete the duplicates and sort
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [41]:
#create the dictionary
work_dict = data[['work_id', 'income_type']]

#delete the duplicates and sort
work_dict = work_dict.drop_duplicates().reset_index(drop=True)
work_dict

,work_id,income_type
0,0,сотрудник
1,2,пенсионер
2,1,компаньон
3,3,госслужащий
4,5,безработный
5,99,студент
6,4,предприниматель
7,6,в декрете


With all the id columns in place and the dictionaries prepared, we can delete the original columns from our table.

In [42]:
data.drop(['education', 'family_status', 'purpose', 'purpose_categorized', 'income_type'], axis=1, inplace=True)

Now we can see if there are any more duplicates.

In [43]:
data.duplicated().sum()

333

Aha! We found more duplicates. Let's remove them and get our final table.

In [44]:
data.drop_duplicates().reset_index(drop=True)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose_id,work_id
0,1,8437,42,0,0,F,0,253875.639453,1,0
1,1,4024,36,1,0,F,0,112080.014102,2,0
2,0,5623,33,1,0,M,0,145885.952297,1,0
3,3,4124,32,1,0,M,0,267628.550329,3,0
4,0,340266,53,1,1,F,0,158616.077870,0,2
5,0,926,27,0,1,M,0,255763.565419,1,1
6,0,2879,43,0,0,F,0,240525.971920,1,1
7,0,152,50,1,0,M,0,135823.934197,3,0
8,2,6929,35,0,1,F,0,95856.832424,0,0
9,0,2188,41,1,0,M,0,144425.938277,1,0


#### Conclusion

We have categorized the data according to the value of `purpose` ("wedding", "real estate", "car", "education", "error"), and assigned IDs to them as `purpose_id`. We have created the following ID dictionaries:

* `purpose_dict` (`purpose` and `purpose_id`)
* `education_dict` (`education` and `education_id`)
* `family_dict` (`family_status` and `family_status_id`)
* `work_dict` (`income_type` and `work_id`)

We have also dropped the `purpose`, `purpose_categorized`, `education`, `family_status`, and `income_type` columns.

Our approach has the following potential problems:

1. This method assumes that a single person did not take out two loans in the same category in one year. For example, if a 52-year old worker takes out a loan to buy commercial real estate, and another loan to repair real estate three months later, this method would treat these two loans as a singular duplicated loan. If we had a loan date column, this problem would not happen. In either case, this is not a common occurence, and should not seriously affect our study.
2. Since we do not have any unique ID for users, not even a name, we still have the problem of two clients the same age. For example, if two 63 year old pensioners took out a loan each in the same category (one for repairs, one for mortgage), this method would treat them as a singular duplicated loan. However, this is a rare occurence and shouldn't affect our study much.

## Client Questions

### Connection Between Children and Paying Off Loans

With this question, as with the rest of them, we cannot guarantee a connection. However, we can demonstrate a correlation. Let's take "there is a correlation between having children and paying off loans late" as our hypothesis.

To test this, we need to compare the ratio of clients who have trouble paying off loans (1 in `debt`) to the overall number of clients. We need to then compare that to similar ratios only for clients with children (`children` >= 0), and only without children (`children` == 0).

Let's create a table only with clients that have children.

In [45]:
data_with_children = data[data['children'] > 0]

data_with_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7209 entries, 0 to 21329
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          7209 non-null   int64  
 1   days_employed     7209 non-null   int64  
 2   dob_years         7209 non-null   int64  
 3   education_id      7209 non-null   int64  
 4   family_status_id  7209 non-null   int64  
 5   gender            7209 non-null   object 
 6   debt              7209 non-null   int64  
 7   total_income      7209 non-null   float64
 8   purpose_id        7209 non-null   int64  
 9   work_id           7209 non-null   int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 619.5+ KB


And group this table according to loan status.

In [46]:
data_with_children.groupby('debt')['work_id'].count()

debt
0    6543
1     666
Name: work_id, dtype: int64

Alright, and now we do the same for clients without children.

In [47]:
data_without_children = data[data['children'] == 0]

data_without_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14021 entries, 2 to 21326
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          14021 non-null  int64  
 1   days_employed     14021 non-null  int64  
 2   dob_years         14021 non-null  int64  
 3   education_id      14021 non-null  int64  
 4   family_status_id  14021 non-null  int64  
 5   gender            14021 non-null  object 
 6   debt              14021 non-null  int64  
 7   total_income      14021 non-null  float64
 8   purpose_id        14021 non-null  int64  
 9   work_id           14021 non-null  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 1.2+ MB


In [48]:
data_without_children.groupby('debt')['work_id'].count()

debt
0    12963
1     1058
Name: work_id, dtype: int64

And our overall ratio.

In [49]:
data.groupby('debt')['work_id'].count()

debt
0    19506
1     1724
Name: work_id, dtype: int64

Excellent. But what about different numbers of children?

In [50]:
for i in [0,1,2,3,4,5]:
    
    #number of kids 
    print('children:', i)
    
    #select and count unpaid loans
    print('unpaid loans:', data[(data['children'] == i) & (data['debt'] == 1)]['work_id'].count())
    
    #total number of loans
    print('total loans:', data[data['children'] == i]['work_id'].count())
    
    #their ratio
    print('ratio:', 100*data[(data['children'] == i) & (data['debt'] == 1)]['work_id'].count()/data[data['children'] == i]['work_id'].count(), '%')
    print('-----')

children: 0
unpaid loans: 1058
total loans: 14021
ratio: 7.545824120961415 %
-----
children: 1
unpaid loans: 441
total loans: 4792
ratio: 9.202838063439065 %
-----
children: 2
unpaid loans: 194
total loans: 2039
ratio: 9.514467876410006 %
-----
children: 3
unpaid loans: 27
total loans: 328
ratio: 8.231707317073171 %
-----
children: 4
unpaid loans: 4
total loans: 41
ratio: 9.75609756097561 %
-----
children: 5
unpaid loans: 0
total loans: 9
ratio: 0.0 %
-----


Let's put it together in a nice table.

In [51]:
#create a list of the number of unpaid loans per child number 
children_unpaid_list = []

#create a list of the total number of loans per child number
children_total_list = []

#populate the lists
for i in [0,1,2,3,4,5]:
    
    #number of unpaid loans for this number of children
    unpaid_number = data[(data['children'] == i) & (data['debt'] == 1)]['work_id'].count()
    
    #total number of loans for this number of children
    total_number = data[data['children'] == i]['work_id'].count()
    
    #append our lists
    children_unpaid_list.append(unpaid_number)
    children_total_list.append(total_number)
    
#put our data into a dataframe
d_child = {'number of children': ['none', 1, 2, 3, 4, 5, 'some', 'total'],
           'unpaid loans' : [children_unpaid_list[0], children_unpaid_list[1], children_unpaid_list[2],
                                   children_unpaid_list[3], children_unpaid_list[4], children_unpaid_list[5],
                                   sum(children_unpaid_list[1:6]), data[data['debt'] == 1]['children'].count()],
          'total loans' : [children_total_list[0], children_total_list[1], children_total_list[2],
                                   children_total_list[3], children_total_list[4], children_total_list[5],
                                   sum(children_total_list[1:6]), data['children'].count()],
          'ratio (%)' : [100*children_unpaid_list[0]/children_total_list[0], 100*children_unpaid_list[1]/children_total_list[1],
                           100*children_unpaid_list[2]/children_total_list[2], 100*children_unpaid_list[3]/children_total_list[3],
                           100*children_unpaid_list[4]/children_total_list[4], 100*children_unpaid_list[5]/children_total_list[5],
                           100*sum(children_unpaid_list[1:6])/sum(children_total_list[1:6]),
                           100*data[data['debt'] == 1]['children'].count()/data['children'].count()]}

#show our pretty table
data_children_final = pd.DataFrame(data=d_child)
data_children_final

,number of children,unpaid loans,total loans,ratio (%)
0,none,1058,14021,7.545824
1,1,441,4792,9.202838
2,2,194,2039,9.514468
3,3,27,328,8.231707
4,4,4,41,9.756098
5,5,0,9,0.000000
6,some,666,7209,9.238452
7,total,1724,21230,8.120584


#### Conclusion

Data shows Our data shows that yes, having kids is directly correlated with a higher rate of loan non-payment. However, as soon as the number of kids hits 3, the unpaid ratio drops nearly to the total level.

### Connection Between Marital Status and Paying Off Loans

As with the previous question, we cannot establish a causal relationship, only a correlation. Our hypothesis is "yes, there is a correlation". To check it, let's group our data according to `family_id` and compare the unpaid to total loan ratios.

In [52]:
for i in [0,1,2,3,4]:
    
    #show id
    print('family_status_id:', i)
    
    #show unpaid loans
    print('unpaid', data[(data['family_status_id'] == i) & (data['debt'] == 1)]['work_id'].count())
    
    #show total loans
    print('total:', data[data['family_status_id'] == i]['work_id'].count())
    
    #show ratio
    print('ratio (%):', 100*data[(data['family_status_id'] == i) & (data['debt'] == 1)]['work_id'].count()/data[data['family_status_id'] == i]['work_id'].count())
    print('-----')

family_status_id: 0
unpaid 923
total: 12212
ratio (%): 7.558139534883721
-----
family_status_id: 1
unpaid 383
total: 4113
ratio (%): 9.311937758327256
-----
family_status_id: 2
unpaid 62
total: 946
ratio (%): 6.553911205073995
-----
family_status_id: 3
unpaid 84
total: 1179
ratio (%): 7.124681933842239
-----
family_status_id: 4
unpaid 272
total: 2780
ratio (%): 9.784172661870503
-----


Excellent. Now let's put it together in a nice dataframe.

In [53]:
#create lists for the table data
family_unpaid_list = []
family_total_list = []
family_name_list = []

#populate the lists
for i in [0,1,2,3,4]:
    
    family_unpaid_list.append(data[(data['family_status_id'] == i) & (data['debt'] == 1)]['work_id'].count())
    family_total_list.append(data[data['family_status_id'] == i]['work_id'].count())
    
    #copy the full family status name from the dictionary while we're at it
    family_name_list.append(family_dict.iat[i,0])
    
#put the data together
d_family = {'marital status' : [family_name_list[0], family_name_list[1], family_name_list[2],
                                   family_name_list[3], family_name_list[4], 'всего'],
           'unpaid loans' : [family_unpaid_list[0], family_unpaid_list[1], family_unpaid_list[2],
                                      family_unpaid_list[3], family_unpaid_list[4], data[data['debt'] == 1]['family_status_id'].count()],
           'total loans': [family_total_list[0], family_total_list[1], family_total_list[2],
                             family_total_list[3], family_total_list[4], data['family_status_id'].count()],
           'ratio (%)': [100*family_unpaid_list[0]/family_total_list[0], 100*family_unpaid_list[1]/family_total_list[1],
                          100*family_unpaid_list[2]/family_total_list[2], 100*family_unpaid_list[3]/family_total_list[3],
                          100*family_unpaid_list[4]/family_total_list[4], 100*data[data['debt'] == 1]['family_status_id'].count()/data['family_status_id'].count()]}

#generate the dataframe
data_family_final = pd.DataFrame(data=d_family)
data_family_final

,marital status,unpaid loans,total loans,ratio (%)
0,женат / замужем,923,12212,7.558140
1,гражданский брак,383,4113,9.311938
2,вдовец / вдова,62,946,6.553911
3,в разводе,84,1179,7.124682
4,не женат / не замужем,272,2780,9.784173
5,всего,1724,21230,8.120584


#### Conclusion

The data shows a correlation between marital status and nonpayment. Widows/widowers, divorcees, and married people pay off their loans more often than average, while single people and people in common law marriages pay them off worse.

### Connection Between Income and Paying Off Loans

As with the other questions, we're looking for a correlation, not a causal link. Our hypothesis is "yes, there is a correlation". Let's break up our clients' incomes into four quartiles.

In [54]:
#calculate the boundaries between categories
#find median
income_halfway_mark = data['total_income'].median()

#find median between min and median
income_1q_mark = data[data['total_income']<=income_halfway_mark]['total_income'].median()

#find median between median and max
income_2q_mark = data[data['total_income']>=income_halfway_mark]['total_income'].median()

In [55]:
#create function to assign an income group
def income_group_creator(row):
    
    #grab the total income
    user_income = row['total_income']
    
    #compare it to the boundaries
    if (user_income >= data['total_income'].min()) & (user_income < income_1q_mark):
        return(0)
    elif (user_income >= income_1q_mark) & (user_income < income_halfway_mark):
        return(1)
    elif (user_income >= income_halfway_mark) & (user_income < income_2q_mark):
        return(2)
    elif (user_income >= income_2q_mark) & (user_income <= data['total_income'].max()):
        return(3)
    else:
        return(99)

In [56]:
#apply the function to the whole table
data['income_group'] = data.apply(income_group_creator, axis=1)

#take a look at the table
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose_id,work_id,income_group
0,1,8437,42,0,0,F,0,253875.639453,1,0,3
1,1,4024,36,1,0,F,0,112080.014102,2,0,1
2,0,5623,33,1,0,M,0,145885.952297,1,0,2
3,3,4124,32,1,0,M,0,267628.550329,3,0,3
4,0,340266,53,1,1,F,0,158616.077870,0,2,2
5,0,926,27,0,1,M,0,255763.565419,1,1,3
6,0,2879,43,0,0,F,0,240525.971920,1,1,3
7,0,152,50,1,0,M,0,135823.934197,3,0,1
8,2,6929,35,0,1,F,0,95856.832424,0,0,0
9,0,2188,41,1,0,M,0,144425.938277,1,0,2


So, we have broken up our clients into four quartiles with the IDs of 0 to 3, with 99 for error.

Let's see what the ratio of unpaid loans to total loans is for each category.

In [57]:
for i in [0,1,2,3]:
    
    #show income group id
    print('income_group:', i)
    
    #show unpaid loan number
    print('unpaid', data[(data['income_group'] == i) & (data['debt'] == 1)]['work_id'].count())
    
    #show total loan number
    print('total:', data[data['income_group'] == i]['work_id'].count())
    
    #show ratio
    print('ratio (%):', 100*data[(data['income_group'] == i) & (data['debt'] == 1)]['work_id'].count()/data[data['income_group'] == i]['work_id'].count())
    print('-----')

income_group: 0
unpaid 430
total: 5365
ratio (%): 8.014911463187325
-----
income_group: 1
unpaid 375
total: 4304
ratio (%): 8.712825278810408
-----
income_group: 2
unpaid 491
total: 5780
ratio (%): 8.494809688581315
-----
income_group: 3
unpaid 428
total: 5781
ratio (%): 7.403563397336101
-----


As expected, the clients with the highest income have the least trouble with paying off loans on time.

Note that our quartiles aren't exactly the same size. This is due to the fact that we're using median values.

Now let's put it together into a nice table.

In [58]:
#create lists for the table
income_unpaid_list = []
income_total_list = []

#populate the lists
for i in [0,1,2,3]:
    income_unpaid_list.append(data[(data['income_group'] == i) & (data['debt'] == 1)]['work_id'].count())
    income_total_list.append(data[data['income_group'] == i]['work_id'].count())
    
#put together the data
d_income = {'quartile' : ['first', 'second', 'third', 'fourth', 'total'],
           'unpaid loans' : [income_unpaid_list[0], income_unpaid_list[1], income_unpaid_list[2],
                                      income_unpaid_list[3], data[data['debt'] == 1]['income_group'].count()],
           'total loans': [income_total_list[0], income_total_list[1], income_total_list[2],
                             income_total_list[3], data['income_group'].count()],
           'ratio (%)': [100*income_unpaid_list[0]/income_total_list[0], 100*income_unpaid_list[1]/income_total_list[1],
                          100*income_unpaid_list[2]/income_total_list[2], 100*income_unpaid_list[3]/income_total_list[3],
                               100*data[data['debt'] == 1]['income_group'].count()/data['income_group'].count()]}

#put it all together into a dataframe
data_income_final = pd.DataFrame(data=d_income)
data_income_final

,quartile,unpaid loans,total loans,ratio (%)
0,first,430,5365,8.014911
1,second,375,4304,8.712825
2,third,491,5780,8.494810
3,fourth,428,5781,7.403563
4,total,1724,21230,8.120584


#### Conclusion

The client's income doesn't correlate much with nonpayment. THe only exception is that clients with a very high income have a better than average nonpayment rate.

### Connection Between Loan Purpose and Paying Off Loans

As with the other questions, we can only show a correlation, rather than a causal relationship. Our hypothesis is "yes, there is a correlation".

We've already broken down the loan purposes into `purpose_id`. So we can go straight into studying the correlation.

In [59]:
for i in [0,1,2,3]:
    
    #show id
    print('purpose_id:', i)
    
    #show unpaid loans
    print('unpaid', data[(data['purpose_id'] == i) & (data['debt'] == 1)]['work_id'].count())
    
    #show total loans
    print('total:', data[data['purpose_id'] == i]['work_id'].count())
    
    #show ratio
    print('ratio (%):', 100*data[(data['purpose_id'] == i) & (data['debt'] == 1)]['work_id'].count()/data[data['purpose_id'] == i]['work_id'].count())
    print('-----')

purpose_id: 0
unpaid 181
total: 2299
ratio (%): 7.872988255763375
-----
purpose_id: 1
unpaid 777
total: 10703
ratio (%): 7.259646827992151
-----
purpose_id: 2
unpaid 397
total: 4258
ratio (%): 9.323626115547205
-----
purpose_id: 3
unpaid 369
total: 3970
ratio (%): 9.294710327455919
-----


Hm, car loans and education loans have noticeably worse repayment ratios. Let's put it into a nice table.

In [60]:
#build lists for the table
purpose_unpaid_list = []
purpose_total_list = []
purpose_name_list = []

#populate lists
for i in [0,1,2,3]:
    purpose_unpaid_list.append(data[(data['purpose_id'] == i) & (data['debt'] == 1)]['work_id'].count())
    purpose_total_list.append(data[data['purpose_id'] == i]['work_id'].count())
    
    #while we're here, add purpose from dictionary
    purpose_name_list.append(purpose_dict.iat[i,0])
    
#combining the data
d_purpose = {'loan purpose' : [purpose_name_list[0], purpose_name_list[1], purpose_name_list[2],
                                   purpose_name_list[3], 'total'],
           'unpaid loans' : [purpose_unpaid_list[0], purpose_unpaid_list[1], purpose_unpaid_list[2],
                                      purpose_unpaid_list[3], data[data['debt'] == 1]['purpose_id'].count()],
           'total loans': [purpose_total_list[0], purpose_total_list[1], purpose_total_list[2],
                             purpose_total_list[3], data['purpose_id'].count()],
           'ratio (%)': [100*purpose_unpaid_list[0]/purpose_total_list[0], 100*purpose_unpaid_list[1]/purpose_total_list[1],
                          100*purpose_unpaid_list[2]/purpose_total_list[2], 100*purpose_unpaid_list[3]/purpose_total_list[3],
                          100*data[data['debt'] == 1]['purpose_id'].count()/data['purpose_id'].count()]}

#put it all together into a dataframe
data_purpose_final = pd.DataFrame(data=d_purpose)
data_purpose_final

,loan purpose,unpaid loans,total loans,ratio (%)
0,wedding,181,2299,7.872988
1,real estate,777,10703,7.259647
2,car,397,4258,9.323626
3,education,369,3970,9.294710
4,total,1724,21230,8.120584


#### Conclusion

The data shows that there is a correlation. Education loans and car loans correlate with worse loan repayment rates.

## Overall Conclusion

In the course of our study we found several positive correlations between greater than average loan nonpayment rate and a number of client factors. The clearest examples are:

* car loans
* education loans
* presense of children
* a lack of a current/former legal partner

We should highlight that our study did not show, nor could it show, a causal link between loan nonpayment and these factors. We have only shown a correlation.

Moreover, some of our results (marital status and number of children) should not even be shown to the client, as making a credit scoring system based on these factors is a banking ethics violation.